In [ ]:
%matplotlib inline
import torch
from d2l import torch as d2l
import torch.nn

In [ ]:
最大似然的原理
认为网络的输出服从均值为真实值,方差未知的正态分布(方差无需知道,因为在计算梯度时,会在正态分布前加上一个log,最终会省去方差)
而对每一个输出和结果和真实label来计算输出的置信度,将所有的判断结果y^的置信度相乘后,得到最大似然的值
最大似然的值越大,就证明网络的拟合效果越好

In [ ]:
softmax计算,就是将全连接层的输出都转化为概率,n为样本个数
具体的公式为 softmax(O) y[i]=exp(o[i])/sum(exp(o[k]),k=1..n)

softmax应用于交叉熵函数时,这里的y^和y都是单个样本的predict和label,shape为(1*d),d为输出的个数
lossfunction(y^,y)=-sum(y[j]*log(y^[j]),j=1..d)


In [ ]:
信息论的熵
分布p的熵(entropy)
这里的log的底数为pi
h[p]=-sum(p[j]log(p[j]),j=1..d)
熵是知道某件事的真实概率后所实际经历的惊异程度

信息论的惊异
当一个数据流内的数据个体都是相同的,那么此时的惊异是很小的,而数据个体之间相差很大时,惊异就会比较大 
香农用 log(1/p(a)),p(a)为某件事的概率

而交叉熵是h(p,q)=-sum(p[j]log(q[j]),j=1..d)
当p和q等同时,交叉熵越小,即惊异值越小

In [ ]:
关于softmax的导数计算

所以在考虑嵌套softmax找out上的时候,就已经考虑到导数计算的问题,要于损失函数相配合
使得损失函数既能体现predict和label的差异,也要容易计算
而softmax和交叉熵损失函数的配合就非常的巧妙


加入一个nn.linear(3,2),out已经嵌套softmax函数,损失函数为交叉熵损失函数,这里是损失函数是计算单个结果的,shape为1*d
在此实例中 loss(y^,y)=-sum(y[j]log(y^[j]),j=1..d)=sum(y[j]log(y^[j]),j=1..d)=sum(y[j]log(exp(o[j]/sum(exp(o[k]),k=1..d))),j=1..d)
=sum(y[j]*log(sum(exp(o[k],k=1..d))),j=1..d)-sum(y[j]*o[j],j=1..d)
=log(sum(exp(o[k],k=1..d)))-sum(y[j]*o[j],j=1..d)

log(x)'=1/x

loss'(o[j])=exp(o[j])/sum(exp(o[k],k=1..d))-y[j]=softmax(o[j])-y[j]

所以对于输出值的梯度就可以计算出来,利用这些梯度,再继续向前计算梯度